In [22]:
import sql_utils
import pandas as pd
from sqlalchemy import create_engine

import sqlite3
import pandas as pd

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import plotly.express as px
import json
from urllib.request import urlopen
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [2]:
# function to calculate proportion of times in a year there was a certan level of air quality

def calc_proportion(level, year):
    '''
    level: string or int. choose from the following
        "good": aqi ranging from 0 to 50
        "moderate": aqi ranging from 51 to 100
        "unhealthy for sensitive groups": aqi ranging from 101 to 150
        "unhealthy": aqi ranging from 151 to 200
        "very unhealthy": aqi ranging from 201 to 300
        "hazardous": aqi of 301 and above
        int: aqi of whatever the int is or above
    year: the year for which we query the data
    '''
    if level == "good":
        lower = 0
    
    elif level == "moderate":
        lower = 51
        
    elif level == "unhealthy for sensitive groups":
        lower = 101
    
    elif level == "unhealthy":
        lower = 151
        
    elif level == "very unhealthy":
        lower = 201
        
    elif level == "hazardous":
        lower = 301
    
    else:
        lower = level
        
    conn = sqlite3.connect('my_database.sqlite')

    df = pd.read_sql_query(f"SELECT aqi, FIPS, county FROM PM25 WHERE year = {year} AND pollutant_standard = 'PM25 24-hour 2012'", conn)

    conn.close()
        
    r = list(df["FIPS"].unique())
    s = list(df["FIPS"].unique())
    j = 0
            
    for i in df["FIPS"].unique():
        s[j] = len(df["aqi"][(lower <= df["aqi"]) & (df["FIPS"] == i)])
        r[j] = len(df[df["FIPS"] == i])
        j += 1
            
    df1 = pd.DataFrame(df["FIPS"].unique(), columns = ["FIPS"])
    df1["values"] = [s / r for s, r in zip(s, r)]
    df1["county"] = list(df["county"].unique())
        
    return(df1)

In [34]:
def data_census(census_table, columns, year):
    '''
    Args:
        census_table: string. the table from the SQL database we want to work with
        columns: list of strings. each string is a column from the SQL database.
        year: int. the year for which we want to plot the data.
    '''
    
    conn = sqlite3.connect('airpandas_1.sqlite')

    df = pd.read_sql_query(f"SELECT {', '.join(columns)}, NAME, FIPS FROM {census_table} WHERE year = {year}", conn)

    conn.close()
    
    df["NAME"] = df["NAME"].str[:-19]
    
    if len(columns) == 1:
        df["values"] = df[columns[0]]
        return(df)
    
    else:
        df["values"] = df.iloc[:,0:len(columns)].sum(axis=1)
        return(df)

In [79]:
cali_choropleth(calc_proportion("unhealthy for sensitive groups", 2021), "values", "proportion", "FIPS", "county")

In [80]:
cali_choropleth(calc_proportion(100, 2018), "values", "proportion ", "FIPS", "county")

In [ ]:
import os

if not os.path.exists("images"):
    os.mkdir("images")

In [ ]:
fig.write_image("images/fig1.png")

In [70]:
def add_aqi_trace(fig, df, variable, variable_name, FIPS, county_name):
    
    fig.add_trace(go.Choropleth(z = df[variable], 
                                geojson=counties, 
                                locations= df[FIPS], 
                                #color=variable,
                                #color_continuous_scale="Viridis",
                                #locationmode="USA-states",
                                #labels={variable: variable_name},
                                #hover_name= county_name,
                                #hover_data={FIPS: False},
                                text=df[county_name],
                                hovertemplate = "<b>County: %{text}</b><br><br>" + "value: %{z}<br>"
                                ),
                  row = 1, 
                  col = 2)
    
    fig.update_geos(
        fitbounds="locations", 
        visible=False
    )

In [69]:
def add_census_trace(fig, df, variable, variable_name, FIPS, county_name):
    
    fig.add_trace(go.Choropleth(z = df[variable], 
                                geojson=counties, 
                                locations= df[FIPS], 
                                #color=variable,
                                #color_continuous_scale="Viridis",
                                #locationmode="USA-states",
                                #labels={variable: variable_name},
                                #hover_name= county_name,
                                #hover_data={FIPS: False},
                                text=df[county_name],
                                hovertemplate = "<b>County: %{text}</b><br><br>" + "value: %{z}<br>",
                                colorbar_x=0.45
                                ),
                  row = 1, 
                  col = 1)
    
    fig.update_geos(
        fitbounds="locations", 
        visible=False
    )

In [71]:
def create_side_comparison(level, year, census_table, census_columns):
    
    '''
    Args:
        level: an int or string giving the level of air quality index or above we want to include in the plot.
        year: int. the year for which we want to plot the data.
        census_table: string. the table from which we want to gather the census variable(s).
        census_columns: list of strings. the columns from the `census_table` we want to select.
    '''
    
    rows = 1
    cols = 2
    fig = make_subplots(
        rows=rows, cols=cols,
        specs = [[{'type': 'choropleth'} for c in np.arange(cols)] for r in np.arange(rows)],
        subplot_titles = ["Census", "Air Quality"]
        )

    add_aqi_trace(fig, calc_proportion(level, year), "values", "proportion", "FIPS", "county")

    add_census_trace(fig, data_census(census_table, census_columns, year), "values", "values", "FIPS", "NAME")
    
    fig.update_layout(coloraxis = {'colorscale':'viridis'})
    
    return(fig)

In [76]:
fig = create_side_comparison(100, 2019, "Race", ["pct_white"])

In [81]:
fig.show()

In [78]:
plotly.offline.plot(fig)

'temp-plot.html'